In [2]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 19.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import pandas as pd
from konlpy.tag import Okt
from gensim import corpora
from gensim.models import CoherenceModel, LdaModel
import re
import gensim.corpora as corpora
import spacy
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import silhouette_score
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Load data
all_questions = pd.read_csv('/content/drive/MyDrive/URP/Questions_old.csv')
all_answers = pd.read_csv('/content/drive/MyDrive/URP/Answers_old.csv')
Namdong_questions = pd.read_csv('/content/drive/MyDrive/URP/regional_data/인천광역시 남동구/questions_120406.csv')
Namdong_answers = pd.read_csv('/content/drive/MyDrive/URP/regional_data/인천광역시 남동구/answers_120406.csv')
Seocho_questions = pd.read_csv('/content/drive/MyDrive/URP/regional_data/서울특별시 서초구/questions_120129.csv')
Seocho_answers = pd.read_csv ('/content/drive/MyDrive/URP/regional_data/서울특별시 서초구/answers_120129.csv')
Dongdaemun_questions = pd.read_csv('/content/drive/MyDrive/URP/regional_data/서울특별시 동대문구/questions_120125.csv')
Dongdaemun_answers = pd.read_csv('/content/drive/MyDrive/URP/regional_data/서울특별시 동대문구/answers_120125.csv')
stop_words = pd.read_csv('/content/drive/MyDrive/URP/stopwords-ko.txt')
# Rename the columns
all_questions.rename(columns={'제목': 'title', '질문': 'question'}, inplace=True)
all_answers.rename(columns ={'답변' : 'answer'}, inplace=True)

In [ ]:
from konlpy.tag import Okt
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
from collections import Counter

class ImprovedTopicAnalyzer:
    def __init__(self, stop_words_df):
        self.okt = Okt()
        self.stopwords = self._load_stopwords(stop_words_df)
        self.generic_terms = {
            '하다', '되다', '이다', '있다', '없다', '않다', '보다',
            '가다', '오다', '많다', '좋다', '알다', '싶다', '같다',
            '그렇다', '이렇다', '저렇다', '어떻다', '그러다', '이러다',
            '주다', '받다', '말다', '계시다', '드리다', '감사하다'
        }

    def _load_stopwords(self, stop_words_df):
        """Convert stopwords dataframe to set and add domain-specific stops"""
        # Basic stopwords from file
        stopwords = set(stop_words_df.iloc[:, 0].values)

        # Add domain-specific stopwords
        domain_stops = {
            # Question/answer indicators
            '질문', '문의', '궁금', '알려주세요', '있나요', '하나요', '답변',
            '안내', '말씀', '참고', '부탁', '드립니다', '해주세요', '감사합니다',

            # Common suffixes and particles
            '이요', '에요', '인데', '인가요', '할까요', '일까요', '는데요',
            '습니다', '입니다', '시다', '었다', '였다', '았다', '아니다',

            # Common temporal expressions
            '오늘', '내일', '모레', '어제', '아까', '방금', '이번',
            '저번', '다음', '이전', '전', '후',

            # Measurement and quantity
            '정도', '약', '대략', '거의', '매우', '너무', '아주',
            '조금', '많이', '약간', '몇', '얼마', '몇몇'
        }

        return stopwords.union(domain_stops)

    def find_optimal_topics(self, corpus, dictionary, texts, start=2, limit=10, step=1):
        """Find optimal number of topics using coherence scores"""
        coherence_scores = []

        for num_topics in range(start, limit, step):
            model = models.LdaModel(
                corpus=corpus,
                id2word=dictionary,
                num_topics=num_topics,
                random_state=42,
                passes=50,
                alpha=0.1,
                eta=0.01
            )

            coherence_model = CoherenceModel(
                model=model,
                texts=texts,
                dictionary=dictionary,
                coherence='c_v'
            )

            coherence_scores.append(coherence_model.get_coherence())

        return start + coherence_scores.index(max(coherence_scores)) * step

    def preprocess_text(self, text):
        """Enhanced text preprocessing"""
        if pd.isna(text):
            return []

        # Extract and filter nouns
        nouns = [
            noun for noun in self.okt.nouns(str(text))
            if len(noun) >= 2
            and noun not in self.stopwords
            and noun not in self.generic_terms
        ]

        # Get important verbs and adjectives
        pos_tags = self.okt.pos(str(text), stem=True)
        important_words = [
            word for word, pos in pos_tags
            if pos in ['Verb', 'Adjective']
            and len(word) >= 2
            and word not in self.stopwords
            and word not in self.generic_terms
            and not any(suffix in word for suffix in ['되다', '하다', '있다', '없다'])
        ]

        # Remove low-frequency terms
        all_words = nouns + important_words
        word_counts = Counter(all_words)
        return [word for word in all_words if word_counts[word] >= 2]

    def analyze_topics(self, data, num_topics=None, min_topics=2, max_topics=10):
        """Analyze topics with automatic topic number selection"""
        # Prepare text data
        if 'title' in data.columns and 'question' in data.columns:
            data['full_text'] = data['title'].fillna('') + ' ' + data['question'].fillna('')
        else:
            data['full_text'] = data['answer'].fillna('')

        # Preprocess documents
        processed_docs = []
        raw_docs = []
        for text in data['full_text']:
            if pd.notna(text):
                tokens = self.preprocess_text(text)
                if len(tokens) > 3:
                    processed_docs.append(tokens)
                    raw_docs.append(text)

        if not processed_docs:
            return None, 0, 0

        # Create dictionary and corpus
        dictionary = corpora.Dictionary(processed_docs)
        dictionary.filter_extremes(no_below=3, no_above=0.5)
        corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

        # Find optimal number of topics if not specified
        if num_topics is None:
            num_topics = self.find_optimal_topics(
                corpus, dictionary, processed_docs,
                start=min_topics, limit=max_topics
            )

        # Train LDA model
        lda_model = models.LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=42,
            passes=100,
            alpha=0.1,
            eta=0.01,
            iterations=2000,
            minimum_probability=0.01
        )

        # Calculate metrics
        topic_vectors = np.zeros((len(corpus), num_topics))
        for i, doc_topics in enumerate(lda_model.get_document_topics(corpus)):
            for topic_id, prob in doc_topics:
                topic_vectors[i, topic_id] = prob

        dominant_topics = np.argmax(topic_vectors, axis=1)
        silhouette_avg = silhouette_score(topic_vectors, dominant_topics, metric='cosine')

        # Extract topics information
        topics_info = []
        for topic_id in range(num_topics):
            # Get top terms
            top_terms = lda_model.show_topic(topic_id, topn=15)

            # Find representative documents
            topic_docs = []
            for doc_id, doc_bow in enumerate(corpus):
                topic_dist = lda_model.get_document_topics(doc_bow)
                doc_topic_probs = dict(topic_dist)

                # Use stricter criteria for document selection
                if (topic_id in doc_topic_probs
                    and doc_topic_probs[topic_id] > 0.4
                    and topic_id == max(doc_topic_probs.items(), key=lambda x: x[1])[0]):
                    topic_docs.append((raw_docs[doc_id], doc_topic_probs[topic_id]))

            # Sort and filter examples
            topic_docs.sort(key=lambda x: x[1], reverse=True)
            topics_info.append({
                'terms': [(term, prob) for term, prob in top_terms if prob >= 0.02],
                'examples': topic_docs[:3],
                'doc_count': len(topic_docs)
            })

        return topics_info, silhouette_avg, num_topics

def print_topic_analysis(topics_info, silhouette_score, content_type="Community"):
    """Print topic analysis with improved formatting"""
    print(f"Silhouette Score: {silhouette_score:.3f}\n")
    print(f"{content_type} Topics:")

    for idx, topic in enumerate(topics_info, 1):
        print(f"\nTopic {idx} (Documents: {topic['doc_count']}):")
        print("Key Terms:", end=" ")
        terms_str = ", ".join([f"{term}({prob:.3f})" for term, prob in topic['terms']])
        print(terms_str)

        if topic['examples']:
            print("\nExample Documents:")
            for text, prob in topic['examples']:
                # Clean and truncate text
                cleaned_text = text.replace('\n', ' ').strip()
                truncated_text = cleaned_text[:150] + "..." if len(cleaned_text) > 150 else cleaned_text
                print(f"- [{prob:.2f}] {truncated_text}")

        print("-" * 80)

def main():
    analyzer = ImprovedTopicAnalyzer(stop_words)

    regions = [
        ('Namdong', Namdong_questions, Namdong_answers),
        ('Seocho', Seocho_questions, Seocho_answers),
        ('Dongdaemun', Dongdaemun_questions, Dongdaemun_answers)
    ]

    for region_name, questions_df, answers_df in regions:
        print(f"\n{'='*80}")
        print(f"Analysis for {region_name} Community")
        print(f"{'='*80}")

        # Analyze questions
        print("\nQUESTION TOPICS:")
        topics_info, silhouette, num_topics = analyzer.analyze_topics(
            questions_df,
            min_topics=5,
            max_topics=7
        )
        print_topic_analysis(topics_info, silhouette, "Question")

        # Analyze answers
        print("\nANSWER TOPICS:")
        topics_info, silhouette, num_topics = analyzer.analyze_topics(
            answers_df,
            min_topics=5,
            max_topics=7
        )
        print_topic_analysis(topics_info, silhouette, "Answer")

if __name__ == "__main__":
    main()


Analysis for Namdong Community

QUESTION TOPICS:
Silhouette Score: 0.833

Question Topics:

Topic 1 (Documents: 6):
Key Terms: 구월동(0.467), 버스(0.350), 인천(0.182)

Example Documents:
- [0.98] 질문구월동 인천종합터미널에서 마산까지 가는 버스 예매 버스타고와 티버스가 있던데요 둘다 구월동 종합터미널을 이용하는 건가요?? 버스타고에 문의하니 상담사 자체가 모른다고 합니다.. 운송회사에서 안다고 하는데 어이없네요. 자기들은 운송회사에서 주는 정보만 올려주기때문에 ...
- [0.89] 질문인천 구월동 구월남로 146 파티마오피스텔 방음 안녕하세요! 인천 구월남로 146 파티마오피스텔(구월동)에 궁굼한 것이 있습니다. 1. 밖에 문소리는 심하게 나는지 궁굼합니다. 2. 옆방 소리 방음은 되는지 궁굼합니다. 3. 고시원 보다는 방음이 좋을까요? 파티마오...
- [0.81] 질문구월동 로데오 10명정도 예약 가능한 술집 있을까요? 구월동 로데오 10명정도 단체 주말에 예약 가능한 술집 있을까요? 노래타운은 빼고요!
--------------------------------------------------------------------------------

Topic 2 (Documents: 11):
Key Terms: 인천(0.674), 추천(0.195), 남동구(0.130)

Example Documents:
- [0.89] 질문인천남동구 빌라 계단청소업체 추천해주세요 !! 4층까지있는 빌라이구요. 인천 만수동. 계딘청소 주1회로 꼼꼼히해주시는 업체있음 추천좀 부탁드려요 지금 계단청소업체는 주1회씩 청소를해주시는데 계단만 빗자루로 휙휙 쓸고 가는거 같은 상태예요. 현관문 구석에 전날에본 죽...
- [0.89] 질문인천 남동구 출산지원 인천 남동구 이번에 넷째아이 임신 입니다. 어떠한 지원이 있을까요?
- [0.89] 

###all_answers (category=6)

In [ ]:
def preprocess_text_korean(text):
    if pd.isna(text) or text is None:
        return ""

    text = str(text).strip()

    # Normalize whitespace
    text = re.sub(r'[\s\u3000\u2000-\u200F\u2028-\u202F]+', ' ', text)

    # Keep Korean, English, numbers, and spaces
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', text)

    # Handle spacing around numbers and English
    text = re.sub(r'([0-9a-zA-Z])([가-힣])', r'\1 \2', text)
    text = re.sub(r'([가-힣])([0-9a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def preprocess_korean(text, stopwords, is_question=False):
    okt = Okt()

    # Define answer-specific stopwords
    answer_stopwords = {
        '답변', '안내', '말씀', '드립니다', '감사합니다', '문의', '내용',
        '확인', '부탁드립니다', '경우', '이용', '관련', '때문', '정도',
        '드리다', '되다', '하다', '있다', '이다', '안내드리다'
    }

    all_stopwords = stopwords.union(answer_stopwords)
    nouns = [n for n in okt.nouns(str(text)) if n not in all_stopwords and len(n) > 1]
    return nouns

def analyze_answer_topics(all_answers, num_topics=6):
    # Clean text
    all_answers['cleaned_text'] = all_answers['answer'].apply(preprocess_text_korean)

    # Process documents
    docs = []
    raw_docs = []
    for text in all_answers['cleaned_text']:
        if pd.notna(text):
            tokens = preprocess_korean(text, stopwords, is_question=False)
            if len(tokens) > 3:  # Only include documents with meaningful content
                docs.append(tokens)
                raw_docs.append(text)

    # Create dictionary and corpus
    dictionary = corpora.Dictionary(docs)
    dictionary.filter_extremes(no_below=5, no_above=0.3)
    corpus = [dictionary.doc2bow(doc) for doc in docs]

    # Train LDA model
    lda_model = models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=20,
        alpha='symmetric'
    )

    # Calculate silhouette score
    doc_topics = [lda_model.get_document_topics(doc, minimum_probability=0.0) for doc in corpus]
    topic_vectors = np.array([[prob for _, prob in sorted(doc)] for doc in doc_topics])
    dominant_topics = np.argmax(topic_vectors, axis=1)
    silhouette_avg = silhouette_score(topic_vectors, dominant_topics, metric='cosine')

    # Extract topics information
    topics_info = []
    for topic_id in range(num_topics):
        top_terms = lda_model.show_topic(topic_id, topn=10)

        topic_docs = []
        for doc_id, doc_bow in enumerate(corpus):
            topic_dist = lda_model.get_document_topics(doc_bow)
            doc_topic_probs = dict(topic_dist)
            if topic_id in doc_topic_probs and doc_topic_probs[topic_id] > 0.4:
                topic_docs.append((raw_docs[doc_id], doc_topic_probs[topic_id]))

        topic_docs.sort(key=lambda x: x[1], reverse=True)
        examples = topic_docs[:10]

        topics_info.append({
            'terms': top_terms,
            'examples': examples
        })

    return topics_info, silhouette_avg

def analyze_all_answers(all_answers):
    print(f"\n{'='*80}")
    print("Analysis of All Answers")
    print(f"{'='*80}")

    topics_info, silhouette_avg = analyze_answer_topics(all_answers)

    print(f"Silhouette Score: {silhouette_avg:.3f}\n")
    print("Answer Topics:\n")

    for idx, topic in enumerate(topics_info, 1):
        print(f"\nTopic {idx}:")
        print("Key Terms:")
        terms = [(term, f"{prob:.3f}") for term, prob in topic['terms']]
        print(", ".join([f"{term}({prob})" for term, prob in terms]))

        print("\nExample Answers:")
        for text, prob in topic['examples']:
            print(f"- [{prob:.2f}] {text[:100]}...")
        print("-" * 80)

# Run the analysis
analyze_all_answers(all_answers)


Analysis of All Answers
Silhouette Score: 0.701

Answer Topics:


Topic 1:
Key Terms:
학교(0.016), 고등학교(0.009), 상담(0.009), 학생(0.009), 학원(0.008), 추천(0.007), 관리(0.006), 생각(0.006), 도움(0.006), 내신(0.006)

Example Answers:
- [1.00] 고교위탁 최다 과정 승인 전국 고교위탁 최다 수강생 안녕하세요 고교위탁 NO 1 인천중앙직업전문학교 입니다 첫 번째 위탁교육은 지역과 상관없이 지원할 수 있기 때문에학생이 원하는 ...
- [0.99] 대학교에 패션디자인과가 있는지 몰랐네요 등급은 모르겠고 실기전형으로 진행되서 발상과표현 기초디자인 선택해서 지원가능하시네요 학교자체가 엄청 높고 그런건 개개인마다 생각차이가 있기때...
- [0.99] 고교위탁 최다 과정 승인 전국 고교위탁 최다 수강생 안녕하세요 고교위탁 No 1 인천중앙직업전문학교 입니다 위탁학교에 대한 정보는 고용노동부 HRD Net 홈페이지를 통해지역별 전...
- [0.99] 최초 4 차산업 특성화 전문교육기관 안녕하세요 고교위탁 NO 1 인천중앙직업전문학교 입니다 위탁교육기관에 대한 정보는 고용 24 HRD Net 홈페이지를 통해 지역별 전공별로 확인...
- [0.99] 고교위탁 최다 과정 승인 전국 고교위탁 최다 수강생 안녕하세요 고교위탁 NO 1 인천중앙직업전문학교 입니다 위탁학교에 관련된 정보는 고용 24 HRD Net 홈페이지를 통해 지역별...
- [0.99] 최초 4 차산업 특성화 전문교육기관 안녕하세요 고교위탁 No 1 인천중앙직업전문학교 입니다 위탁학교마다 다르기 때문에해당 기관에 문의하는 것이 가장 정확합니다 위탁교육은 진로에 있...
- [0.99] 8 학군 여고에서 3 등급을 받는 것은 전국적으로는 상위권 성적에 해당하나 강남 8 학군의 경쟁이 치열한 만큼 학교 내에서 중위권으로 평가될 수 있습니다 3 등급은 대학 입시

###all_questions(category=6)

In [6]:
# Step 1: Define a preprocessing function for cleaning Korean text
def preprocess_text_korean(text):
    """
    Preprocess Korean text by:
    - Removing extra spaces
    - Keeping only Korean, English, and spaces
    """
    if pd.isna(text):  # Handle NaN values
        return ""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z\s]', ' ', text)  # Retain Korean, English, and spaces only
    return text.strip()  # Remove leading/trailing whitespace

# Step 2: Combine title and question columns for preprocessing
# Assuming 'title' and 'question' are the relevant columns in all_questions
all_questions['combined_text'] = all_questions['title'].fillna('') + " " + all_questions['question'].fillna('')

# Step 3: Apply text preprocessing
all_questions['cleaned_text'] = all_questions['combined_text'].apply(preprocess_text_korean)

# Step 4: Define tokenization function using Okt tokenizer
from konlpy.tag import Okt

def preprocess_korean(text):
    """
    Tokenize Korean text into nouns while removing common stopwords.
    """
    okt = Okt()  # Initialize the Okt tokenizer
    # Define common stopwords
    common_stopwords = {'질문', '곳', '제', '요', '좀', '어디', '저', '분', '수', '중',
                        '정도', '시', '있나요', '알려주세요', '있을까요', '하는데', '혹시'}
    # Tokenize text into nouns and filter stopwords and single-character tokens
    nouns = [n for n in okt.nouns(str(text)) if n not in common_stopwords and len(n) > 1]
    return nouns

# Step 5: Analyze community topics using LDA
from gensim import corpora, models
from sklearn.metrics import silhouette_score
import numpy as np

def analyze_community_topics(data, num_topics=6):
    """
    Perform LDA topic modeling on the community questions and calculate a silhouette score.

    Steps:
    - Tokenize text into nouns
    - Create a dictionary and bag-of-words corpus
    - Train an LDA model
    - Calculate the silhouette score to evaluate topic separability
    - Extract top terms and example documents for each topic
    """
    # Step 5.1: Preprocess text for topic modeling
    docs, raw_docs = [], []
    for text in data['cleaned_text']:
        if pd.notna(text):
            tokens = preprocess_korean(text)
            if len(tokens) > 3:
                docs.append(tokens)
                raw_docs.append(text)

    # Step 5.2: Create dictionary and corpus for LDA
    dictionary = corpora.Dictionary(docs)
    dictionary.filter_extremes(no_below=5, no_above=0.3)
    corpus = [dictionary.doc2bow(doc) for doc in docs]

    # Step 5.3: Train the LDA model
    lda_model = models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=20,
        alpha='symmetric'
    )

    # Step 5.4: Calculate silhouette score
    doc_topics = [lda_model.get_document_topics(doc, minimum_probability=0.0) for doc in corpus]
    topic_vectors = np.array([[prob for _, prob in sorted(doc)] for doc in doc_topics])
    dominant_topics = np.argmax(topic_vectors, axis=1)
    silhouette_avg = silhouette_score(topic_vectors, dominant_topics, metric='cosine')

    # Step 5.5: Extract topics and example documents
    topics_info = []
    for topic_id in range(num_topics):
        top_terms = lda_model.show_topic(topic_id, topn=10)
        topic_docs = [
            (raw_docs[doc_id], dict(lda_model.get_document_topics(corpus[doc_id]))[topic_id])
            for doc_id in range(len(corpus))
            if topic_id in dict(lda_model.get_document_topics(corpus[doc_id]))
            and dict(lda_model.get_document_topics(corpus[doc_id]))[topic_id] > 0.4
        ]
        topic_docs = sorted(topic_docs, key=lambda x: x[1], reverse=True)[:10]
        topics_info.append({
            'topic_id': topic_id,
            'terms': top_terms,
            'examples': topic_docs
        })

    return topics_info, silhouette_avg

# Step 6: Run topic analysis on all_questions
topics_info, silhouette_score = analyze_community_topics(all_questions)

# Step 7: Display the results
print(f"Silhouette Score: {silhouette_score:.3f}\n")
print("Community Question Topics:\n")
for idx, topic in enumerate(topics_info, 1):
    print(f"\nTopic {idx}:")
    print("Key Terms:")
    terms = [(term, f"{prob:.3f}") for term, prob in topic['terms']]
    print(", ".join([f"{term}({prob})" for term, prob in terms]))

    print("\nExample Questions:")
    for text, prob in topic['examples']:
        print(f"- [{prob:.2f}] {text[:100]}...")
    print("-" * 80)

KeyError: 0